In [1]:
import os

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.client_tool import client_tool

import rich
from rich.pretty import pprint

from dotenv import load_dotenv
load_dotenv()

HOST=os.environ["HOST"]
PORT=os.environ["LLAMA_STACK_PORT"]
MODEL_NAME=os.environ["INFERENCE_MODEL"]



In [2]:
import requests

@client_tool
def weathertool(latitude: float, longitude: float) -> float:
    """
    Get current temperature for provided coordinates in celsius

    :param latitude: latitude of the location
    :param longitude: longitude of the location
    :returns: value containing the temparature of the provided coordinates in celsius
    """
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    
    return data['current']['temperature_2m']

In [3]:
def agent_example():
    client = LlamaStackClient(base_url=f"http://{HOST}:{PORT}")
    agent = Agent(
        client, 
        model=MODEL_NAME,
        instructions= "Give an exact answer to the question asked. Use tools for providing answers when needed. Do not use tools if not needed. Think carefully before answering. If the question is complex, break it down into simple steps and execute",
        sampling_params={
        "strategy": {"type": "top_p", "temperature": 0.7, "top_p": 0.95},
        "max_tokens": 2048,
        },
        tools=[weathertool],
    )

    return agent
    


In [4]:
agent=agent_example()
session_id = agent.create_session("test-tool_call_session")
print(f"Created session_id={session_id} for Agent({agent.agent_id})")


# "I am planning a trip to Switzerland, what are the top 3 places to visit?",
#  "what is the temparature in Paris?",
# "When was the last time that the Boston Red Sox won the world series?"


response = agent.create_turn(
    messages=[
        {
            "role": "user",
            "content": "I am planning a trip to Switzerland, what are the top 3 places to visit?",
        }
    ],
    session_id=session_id,
    stream=False
)
rich.print(f"[bold yellow]Agent Answer:[/bold yellow] {response.output_message.content}")
pprint(response)
#print(response.__dict__)  # Shows the internal dictionary

Created session_id=7488b146-c11f-4497-bf5c-baaf50bfad44 for Agent(19dbecaf-8283-44a5-827f-fc12e6d4cbcc)


Agent Answer: Based on general information and assuming you don't mind the current weather conditions, here are 
three of the top places to visit in Switzerland:

1. **Jungfraujoch**: Located in the Bernese Oberland region, Jungfraujoch is often referred to as the "Top of 
Europe." It's a mountain peak with breathtaking views of the surrounding Alps and glaciers. Visitors can enjoy the 
unique Ice Palace, Alpine Sensory World, and take in the stunning vistas from the observation decks.
2. **Lake Geneva**: Situated in western Switzerland, Lake Geneva (Lac Léman) is the largest lake in the country and
a popular destination for boat cruises, water sports, and relaxing strolls along its picturesque shores. The city 
of Geneva is also nearby, with its rich history, cultural attractions, and international organizations such as the 
United Nations and the Red Cross.
3. **Interlaken**: Located in the Bernese Oberland region, Interlaken is a beautiful town surrounded by mountains 
and lakes. It's an ideal base for outdoor activities like hiking, paragliding, canyoning, and rafting. Visitors can
also take scenic boat rides on Lake Thun or Lake Brienz, or explore the nearby Trümmelbach Falls.

These are just a few of the many amazing destinations in Switzerland. Depending on your interests and the time of 
year you visit, there are plenty of other places to explore!

Turn(
│   input_messages=[
│   │   UserMessage(
│   │   │   content='I am planning a trip to Switzerland, what are the top 3 places to visit?',
│   │   │   role='user',
│   │   │   context=None
│   │   ),
│   │   ToolResponseMessage(
│   │   │   call_id='7827c959-0da4-40ce-b8e5-626533e84e2d',
│   │   │   content='10.6',
│   │   │   role='tool',
│   │   │   tool_name='weathertool'
│   │   )
│   ],
│   output_message=CompletionMessage(
│   │   content='Based on general information and assuming you don\'t mind the current weather conditions, here are three of the top places to visit in Switzerland:\n\n1. **Jungfraujoch**: Located in the Bernese Oberland region, Jungfraujoch is often referred to as the "Top of Europe." It\'s a mountain peak with breathtaking views of the surrounding Alps and glaciers. Visitors can enjoy the unique Ice Palace, Alpine Sensory World, and take in the stunning vistas from the observation decks.\n2. **Lake Geneva**: Situated in western Switzerland, Lake Geneva (Lac Léman) is the largest lake in the country and a popular destination for boat cruises, water sports, and relaxing strolls along its picturesque shores. The city of Geneva is also nearby, with its rich history, cultural attractions, and international organizations such as the United Nations and the Red Cross.\n3. **Interlaken**: Located in the Bernese Oberland region, Interlaken is a beautiful town surrounded by mountains and lakes. It\'s an ideal base for outdoor activities like hiking, paragliding, canyoning, and rafting. Visitors can also take scenic boat rides on Lake Thun or Lake Brienz, or explore the nearby Trümmelbach Falls.\n\nThese are just a few of the many amazing destinations in Switzerland. Depending on your interests and the time of year you visit, there are plenty of other places to explore!',
│   │   role='assistant',
│   │   stop_reason='end_of_turn',
│   │   tool_calls=[]
│   ),
│   session_id='7488b146-c11f-4497-bf5c-baaf50bfad44',
│   started_at=datetime.datetime(2025, 4, 15, 2, 31, 2, 215894, tzinfo=TzInfo(-04:00)),
│   steps=[
│   │   InferenceStep(
│   │   │   api_model_response=CompletionMessage(
│   │   │   │   content='',
│   │   │   │   role='assistant',
│   │   │   │   stop_reason='end_of_turn',
│   │   │   │   tool_calls=[
│   │   │   │   │   ToolCall(
│   │   │   │   │   │   arguments={'latitude': 47.5236, 'longitude': 8.3528},
│   │   │   │   │   │   call_id='7827c959-0da4-40ce-b8e5-626533e84e2d',
│   │   │   │   │   │   tool_name='weathertool',
│   │   │   │   │   │   arguments_json=None
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   step_id='b51a2847-fca1-497b-bcb7-4b78c6cb9d2b',
│   │   │   step_type='inference',
│   │   │   turn_id='bde25f52-89e7-439e-a46a-fb3d1b54b22b',
│   │   │   completed_at=datetime.datetime(2025, 4, 15, 2, 31, 11, 245737, tzinfo=TzInfo(-04:00)),
│   │   │   started_at=datetime.datetime(2025, 4, 15, 2, 31, 2, 216020, tzinfo=TzInfo(-04:00))
│   │   ),
│   │   ToolExecutionStep(
│   │   │   step_id='98f403e1-3ecb-49e7-958e-bd4bec5b6f4c',
│   │   │   step_type='tool_execution',
│   │   │   tool_calls=[
│   │   │   │   ToolCall(
│   │   │   │   │   arguments={'latitude': 47.5236, 'longitude': 8.3528},
│   │   │   │   │   call_id='7827c959-0da4-40ce-b8e5-626533e84e2d',
│   │   │   │   │   tool_name='weathertool',
│   │   │   │   │   arguments_json=None
│   │   │   │   )
│   │   │   ],
│   │   │   tool_responses=[
│   │   │   │   ToolResponse(
│   │   │   │   │   call_id='7827c959-0da4-40ce-b8e5-626533e84e2d',
│   │   │   │   │   content='10.6',
│   │   │   │   │   tool_name='weathertool',
│   │   │   │   │   metadata={}
│   │   │   │   )
│   │   │   ],
│   │   │   turn_id='bde25f52-89e7-439e-a46a-fb3d1b54b22b',
│   │   │   completed_at=datetime.datetime(2025, 4, 15, 2, 31, 11, 826479, tzinfo=TzInfo(-04:00)),
│   │   │   started_at=datetime.datetime(2025, 4, 15, 2, 31, 11, 280812, tzinfo=TzInfo(-04:00))
│   │   ),
│   │   InferenceStep(
│   │   │   api_model_response=Comple